<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine_tuning_mixed/ask_2_mixed_pubmed_cadec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
curl: (60) SSL certificate problem: certificate has expired
More details here: https://curl.haxx.se/docs/sslcerts.html

curl failed to verify the legitimacy of the server and therefore could not
establish a secure connection to it. To learn more about this situation and
how to fix it, please visit the web page mentioned above.


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-2.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-2.validation.csv",header=None)
valid.head()

,0,1
0,2,left heel pain
1,4,Discoloration of skin [blotchy] on the neck
2,7,goiter
3,8,arthritis
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-2.test.csv",header=None)
test.head()

,0,1
0,2,right heel pain
1,4,black and blue marks
2,8,arthritice
3,8,arthritis
4,8,arthritis


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine_tuning_language_model_mixed.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.217360,4.650884,0.236549,17:37


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.512972,4.361013,0.264538,17:36


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.221818,4.326222,0.267595,17:33


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.177899,4.301391,0.270070,17:37


In [0]:
learn.save_encoder('bigtweet_fold0_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('bigtweet_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.802475,3.180062,0.456464,01:12


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.567790,2.640119,0.580475,01:27


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.219973,2.349478,0.631926,03:48


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.861712,2.095755,0.662269,04:02
1,1.684946,1.997613,0.670185,03:41


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.476554,1.856770,0.689974,04:00
1,1.277104,1.762704,0.709763,03:40


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.231116,1.724461,0.716359,03:56
1,1.089948,1.616487,0.732190,03:49


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.049691,1.534200,0.746702,03:50
1,0.940550,1.466370,0.757256,03:58


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.862201,1.472435,0.757256,03:37
1,0.791688,1.390674,0.774406,04:00


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.752704,1.390976,0.769129,03:50
1,0.759630,1.314482,0.781003,03:40
2,0.681721,1.328017,0.795515,03:56
3,0.598300,1.307102,0.796834,03:56


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.601427,1.309024,0.792876,03:47
1,0.604334,1.286536,0.795515,03:49
2,0.533010,1.295925,0.798153,03:59
3,0.495411,1.326446,0.792876,03:49


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.506129,1.318273,0.808707,03:59
1,0.531025,1.305315,0.796834,03:47
2,0.498948,1.261208,0.803430,03:51
3,0.443921,1.262962,0.803430,03:51


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.472774,1.291052,0.811346,03:45
1,0.500250,1.351381,0.804749,03:38
2,0.467791,1.273536,0.813984,03:50
3,0.387582,1.271780,0.813984,03:38


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.418219,1.315978,0.807388,03:57
1,0.457549,1.307814,0.808707,03:54
2,0.427036,1.279989,0.817942,04:06
3,0.376862,1.249080,0.821900,03:47


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.425092,1.393870,0.815303,03:38
1,0.425168,1.326275,0.811346,04:05
2,0.416067,1.344301,0.807388,03:40
3,0.374328,1.343915,0.812665,03:57


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.373933,1.370091,0.803430,03:56
1,0.422815,1.371291,0.813984,03:41
2,0.377363,1.293481,0.820580,03:56
3,0.343679,1.353774,0.815303,03:52


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-bigtweet')

# Part three: Predict on the test dataset

In [42]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [43]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 262, 933, 8, 8, 8, 933, 8, 764, 16, 118, 27, 30, 705, 584, 584, 584, 32, 197, 37, 37, 861, 37, 37, 37, 37, 37, 37, 37, 37, 39, 385, 44, 477, 49, 269, 49, 49, 780, 100, 100, 100, 100, 100, 100, 408, 52, 53, 785, 785, 53, 640, 58, 785, 62, 62, 815, 1028, 578, 70, 205, 71, 71, 71, 597, 76, 289, 289, 289, 172, 329, 526, 260, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 260, 260, 815, 289, 289, 289, 265, 80, 80, 80, 80, 80, 80, 80, 486, 83, 268, 853, 92, 92, 517, 94, 94, 94, 318, 318, 94, 94, 94, 94, 94, 96, 761, 594, 809, 100, 100, 100, 100, 108, 108, 108, 523, 275, 111, 111, 111, 111, 289, 289, 214, 831, 125, 269, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 289, 131, 132, 135, 540, 540, 137, 388, 142, 143, 269, 151, 151, 152, 714, 157, 161, 162, 163, 166, 166, 172, 864, 864, 864, 864, 604, 289, 683, 180, 182, 182, 289, 289, 183, 183, 337, 187, 187, 187, 187, 187, 189, 190, 190, 190, 190, 190,

In [44]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

585
0.6755196304849884
